In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen
from algorithms import Falsone17DualDecomp

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 10
num_edge = 25

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)

Graph with 10 nodes and 25 edges
(1, 10, 10)


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


In [2]:
from problems import VNResAlloc

prob = VNResAlloc(num_node, d=5)
prob.gen()
# prob.load()


generating a VN resource allocation problem
generated problem saved in data/problem/VNResAlloc/N10_d5


In [3]:
MAX_ITER = 1000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# obj_err_list = []
# cons_val_list = []
# for algo_idx in range(len(algo_list)):
    # algo = algo_list[algo_idx]

alg = Falsone17DualDecomp(prob, network, beta=4.4)
# logging.info(f'{labels[algo_idx]} is running:')
obj_err_log = []
cons_vio_log = []
obj_err, cons_vio = alg.compute_metrics(opt_val=prob.opt_val)
obj_err_log.append(obj_err)
cons_vio_log.append(cons_vio)
for t in range(MAX_ITER):
    alg.step()
    obj_err, cons_vio = alg.compute_metrics(opt_val=prob.opt_val)
    obj_err_log.append(obj_err)
    cons_vio_log.append(cons_vio)
    logging.info(f'iter {alg.iter_num}, obj err: {obj_err:.2e}, cons vio: {cons_vio}')
# obj_err_list.append(obj_err_log)
# cons_val_list.append(cons_vio_log)


iter 1, obj err: 2.13e+00, cons vio: 25.033983397550703
iter 2, obj err: 2.51e+00, cons vio: 15.590517017887668
iter 3, obj err: 2.41e+00, cons vio: 9.78001581612685
iter 4, obj err: 2.25e+00, cons vio: 6.97741543718304
iter 5, obj err: 2.08e+00, cons vio: 5.429085809656412
iter 6, obj err: 1.94e+00, cons vio: 4.428264160896596
iter 7, obj err: 1.82e+00, cons vio: 3.715491773618279
iter 8, obj err: 1.72e+00, cons vio: 3.3083559108937357
iter 9, obj err: 1.63e+00, cons vio: 2.9787755169902246
iter 10, obj err: 1.55e+00, cons vio: 2.7060329863763135
iter 11, obj err: 1.48e+00, cons vio: 2.476244755379078
iter 12, obj err: 1.42e+00, cons vio: 2.279746804288969
iter 13, obj err: 1.37e+00, cons vio: 2.109598020081304
iter 14, obj err: 1.32e+00, cons vio: 1.9606800940032618
iter 15, obj err: 1.28e+00, cons vio: 1.8291332579135429
iter 16, obj err: 1.24e+00, cons vio: 1.7119899576889475
iter 17, obj err: 1.21e+00, cons vio: 1.6069297935868871
iter 18, obj err: 1.17e+00, cons vio: 1.5121112613

In [ ]:
np.savetxt(f'{log_dir}/obj_err_{alg.name}.txt', obj_err_log, delimiter=',')

# print(cons_vio_log)
np.savetxt(f'{log_dir}/cons_vio_{alg.name}.txt', cons_vio_log, delimiter=',')

[35.08953719654275, 25.033983397550703, 15.590517017887668, 9.78001581612685, 6.97741543718304, 5.429085809656412, 4.428264160896596, 3.715491773618279, 3.3083559108937357, 2.9787755169902246, 2.7060329863763135, 2.476244755379078, 2.279746804288969, 2.109598020081304, 1.9606800940032618, 1.8291332579135429, 1.7119899576889475, 1.6069297935868871, 1.5121112613165533, 1.4260535567044401, 1.3517892942206342, 1.3168177809220871, 1.2846174458691781, 1.2548571658026666, 1.227257379281736, 1.20158034827771, 1.17762255651128, 1.1552087152935657, 1.1341869926121109, 1.1144251829216372, 1.0958076074535177, 1.078232586983341, 1.061610366993947, 1.0458614031765283, 1.0309149360652288, 1.0167077992770992, 1.0031834177204428, 0.9902909612267712, 0.9779846203742613, 0.9662230000186961, 0.9549685956610304, 0.9441873422571411, 0.9338482245865294, 0.9239229397019874, 0.9143856035171987, 0.9052124949231374, 0.8963818319284121, 0.8878735752236461, 0.8796692553118176, 0.8717518199563197, 0.864105499201877

In [5]:
from plot_utils import MyFigure

cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $t$', 
                            ylabel=r'$\sum_{i=1}^N g_i(\bar{x}_{i,t})$',
                            yscale='symlog')
cons_vio_figure.add_line("Falsone 17 DD", cons_vio_log)
cons_vio_figure.paint(MAX_ITER=MAX_ITER)

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $t$', 
                            ylabel=r'$\left\vert\sum_{i=1}^N f_i(\bar{x}_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)\right\vert$')
obj_err_figure.add_line("Falsone 17 DD", obj_err_log)
obj_err_figure.paint(MAX_ITER=MAX_ITER)